In [1]:
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
from transformers import Trainer, EvalPrediction, DataCollatorWithPadding, TrainingArguments
from tqdm import tqdm
import numpy as np
import datasets
import torch
from torch.utils.data import DataLoader
# from myutils.evaluate import evaluate_model

In [2]:
pre_pruning_model_name = "/home/meissner/had_experiments/checkpoints/base-models/bert-base-smnli"
# pre_pruning_model_name = "models/pruned/bert-base-snli-prune-layer11-head1"
post_pruning_model_name = "models/pruned/bert-base-snli-prune-layer11-head1"
# pre_pruning_model = BertForSequenceClassification.from_pretrained(pre_pruning_model_name)
post_pruning_model = BertForSequenceClassification.from_pretrained(post_pruning_model_name)
tokenizer = BertTokenizer.from_pretrained(pre_pruning_model_name)

In [3]:
snli = datasets.load_dataset("snli")

Reusing dataset snli (/home/meissner/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
# Prepare eval dataset
def tokenize_func(example):
    return tokenizer(example['premise'], 
                     example['hypothesis'], 
                     padding='max_length', 
                     max_length=128, 
                     truncation=True)
snli = snli.filter(lambda example: example['label'] != -1)
snli_val, snli_test = [sett.map(tokenize_func) for sett in (snli['validation'], snli['test'])]

Loading cached processed dataset at /home/meissner/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b/cache-8ff58c0e12d296bd.arrow
Loading cached processed dataset at /home/meissner/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b/cache-30a438dd3500a0ed.arrow
Loading cached processed dataset at /home/meissner/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b/cache-fab51b5333cb6b5f.arrow
Loading cached processed dataset at /home/meissner/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b/cache-4a6d0e429546f24b.arrow
Loading cached processed dataset at /home/meissner/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b/cache-cb77eff3de0b2266.arrow


In [5]:
dev_not_indices = [index for index, sample in enumerate(snli_val) if ' not ' in sample['hypothesis']]
len(dev_not_indices)

58

In [6]:
test_not_indices = [index for index, sample in enumerate(snli_test) if ' not ' in sample['hypothesis']]
len(test_not_indices)

55

In [7]:
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

preds = None
def evaluate_model(model, tokenizer, dataset):
    model.eval()
    train_args = TrainingArguments(output_dir='temp', per_device_eval_batch_size=64)
    trainer = Trainer(args=train_args,
                    model=model,
                    tokenizer=tokenizer,
                    train_dataset=None,
                    eval_dataset=dataset,
                    compute_metrics=compute_metrics,
                    data_collator=None)

    return trainer.evaluate()

In [8]:
# evaluate_model(pre_pruning_model, tokenizer, snli_val)['eval_accuracy']

In [9]:
# evaluate_model(pre_pruning_model, tokenizer, snli_val.select(dev_not_indices))['eval_accuracy']

In [10]:
# del pre_pruning_model
# torch.cuda.empty_cache()
evaluate_model(post_pruning_model, tokenizer, snli_val)['eval_accuracy']

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: premise, hypothesis.
***** Running Evaluation *****
  Num examples = 9842
  Batch size = 64


RuntimeError: CUDA out of memory. Tried to allocate 1.12 GiB (GPU 0; 31.75 GiB total capacity; 28.34 GiB already allocated; 875.50 MiB free; 29.45 GiB reserved in total by PyTorch)

In [ ]:
evaluate_model(post_pruning_model, tokenizer, snli_val.select(dev_not_indices))['eval_accuracy']